<a href="https://colab.research.google.com/github/Giffy/fast.ai/blob/master/Machine%20Learning/lesson3_grocery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google Colab setup
Installs fast.ai 0.7.0 and the required libraries to run the notebook.

Also downloads the required datasets.

In [0]:
print (" Installing FastAI libraries ... (takes 2 min)")
!pip install fastai==0.7.0 > /dev/null
print ("\n Clonning FastAI repository locally ...")
!git clone https://github.com/fastai/fastai.git fastai_ml
!ln -s fastai_ml/courses/ml1/fastai/ fastai

In [0]:
print ("\n Installing required libraries...")
!pip install --upgrade setuptools > /dev/null
!pip install feather > /dev/null
!pip install scikit-misc==0.1.0 > /dev/null
!pip install pdpbox==0.2.0 > /dev/null
!pip install treeinterpreter==0.2.2 > /dev/null
print ("\n Downloading datasets...")
!wget https://raw.githubusercontent.com/Giffy/Personal_dataset_repository/master/grocery-sales-train.7z
!7z x -y grocery-sales-train.7z > /dev/null
!wget https://raw.githubusercontent.com/Giffy/Personal_dataset_repository/master/grocery-sales-test.tar.gz
!tar xvf grocery-sales-test.tar.gz > /dev/null
print ("\n Import libraries")
import pandas as pd
import os
import numpy as np

# 1 Imports 

In [48]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [0]:
from fastai.imports import *
from fastai.structured import *

from sklearn.ensemble import RandomForestRegressor
from IPython.display import display

from sklearn import metrics

In [61]:
PATH = 'grocery-sales/'
!ls {PATH}

test.csv  train_basic.csv


# 2 Read data

In [0]:
types = {
    'id': 'int64',
    'item_nbr': 'int32',
    'store_nbr': 'int8',
    'unit_sales': 'float32',
    'onpromotion': 'object'
}

In [34]:
%%time
df_all = pd.read_csv(f'{PATH}train_basic.csv', parse_dates=['date'], dtype=types, 
                     infer_datetime_format=True)#, skiprows=range(1,100000000))

CPU times: user 2.65 s, sys: 97.9 ms, total: 2.75 s
Wall time: 2.75 s


In [36]:
df_all.onpromotion.fillna(False, inplace=True)
df_all.onpromotion = df_all.onpromotion.map({'False': False, 'True': True})
df_all.onpromotion = df_all.onpromotion.astype(bool)

os.makedirs('tmp', exist_ok=True)
%time df_all.to_feather('tmp/raw_groceries')

CPU times: user 89.6 ms, sys: 67.5 ms, total: 157 ms
Wall time: 418 ms


In [37]:
%time df_all.describe(include='all')

CPU times: user 850 ms, sys: 1.33 ms, total: 852 ms
Wall time: 1.02 s


,Unnamed: 0,id,date,store_nbr,item_nbr,unit_sales,onpromotion
count,1.997040e+06,1.997040e+06,1997040,1.997040e+06,1.997040e+06,1.997040e+06,1997040
unique,NaN,NaN,1684,NaN,NaN,NaN,2
top,NaN,NaN,2017-05-07 00:00:00,NaN,NaN,NaN,False
freq,NaN,NaN,1908,NaN,NaN,NaN,1528448
first,NaN,NaN,2013-01-01 00:00:00,NaN,NaN,NaN,NaN
last,NaN,NaN,2017-08-15 00:00:00,NaN,NaN,NaN,NaN
mean,2.449852e+07,6.271723e+07,NaN,2.745692e+01,9.728621e+05,8.551911e+00,NaN
std,5.764959e+05,3.619772e+07,NaN,1.633284e+01,5.202940e+05,2.057796e+01,NaN
min,2.350000e+07,2.060000e+02,NaN,1.000000e+00,9.699500e+04,-1.768000e+03,NaN
25%,2.399926e+07,3.138601e+07,NaN,1.200000e+01,5.227210e+05,2.000000e+00,NaN


In [43]:
df_test = pd.read_csv(f'{PATH}test.csv',parse_dates=['date'], dtype=types, infer_datetime_format=True)

df_test.onpromotion.fillna(False, inplace=True)
df_test.onpromotion = df_all.onpromotion.map({'False': False, 'True': True})
df_test.onpromotion = df_all.onpromotion.astype(bool)
df_test.describe(include='all')

,id,date,store_nbr,item_nbr,onpromotion
count,3.370464e+06,3370464,3.370464e+06,3.370464e+06,1997040
unique,NaN,16,NaN,NaN,2
top,NaN,2017-08-27 00:00:00,NaN,NaN,False
freq,NaN,210654,NaN,NaN,1528448
first,NaN,2017-08-16 00:00:00,NaN,NaN,NaN
last,NaN,2017-08-31 00:00:00,NaN,NaN,NaN
mean,1.271823e+08,NaN,2.750000e+01,1.244798e+06,NaN
std,9.729693e+05,NaN,1.558579e+01,5.898362e+05,NaN
min,1.254970e+08,NaN,1.000000e+00,9.699500e+04,NaN
25%,1.263397e+08,NaN,1.400000e+01,8.053210e+05,NaN


In [44]:
df_all.tail()

,Unnamed: 0,id,store_nbr,item_nbr,unit_sales,onpromotion,Year,Month,Week,Day,Dayofweek,Dayofyear,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start,Elapsed
1997035,25497035,73734366,54,1464238,3.135494,False,2016,3,11,18,4,78,False,False,False,False,False,False,1458259200
1997036,25497036,101679247,49,1489899,3.401197,True,2016,12,52,31,5,366,True,False,True,False,True,False,1483142400
1997037,25497037,124420242,38,849080,1.098612,False,2017,8,31,5,5,217,False,False,False,False,False,False,1501891200
1997038,25497038,35980557,31,1463798,1.386294,False,2014,11,48,25,1,329,False,False,False,False,False,False,1416873600
1997039,25497039,85369289,28,362035,2.708050,False,2016,7,29,18,0,200,False,False,False,False,False,False,1468800000


In [0]:
df_all.unit_sales = np.log1p(np.clip(df_all.unit_sales, 0, None))

In [0]:
%time add_datepart(df_all, 'date')

In [51]:
def split_vals(a, n): return a[:n].copy(), a[n:].copy()

n_valid = len(df_test)
n_trn = len(df_all) - n_valid
train, valid = split_vals(df_all, n_trn)
train.shape, valid.shape

((623616, 19), (1373424, 19))

In [52]:
%%time
trn, y, nas = proc_df(train, 'unit_sales')
val, y_val, nas = proc_df(valid, 'unit_sales')

CPU times: user 1.1 s, sys: 450 ms, total: 1.55 s
Wall time: 1.55 s


# 3 Models

In [0]:
def rmse(x, y): return math.sqrt(((x-y)**2).mean())

def print_score(m):
    res = [rmse(m.predict(x), y), rmse(m.predict(val), y_val),
          m.score(x, y), m.score(val, y_val)]
    if hasattr(m, 'oob_score_'): res.append(m.oob_score_)
    print(res)

In [0]:
set_rf_samples(1000000)

In [55]:
%time x = np.array(trn, dtype=np.float32)

CPU times: user 604 ms, sys: 60 ms, total: 664 ms
Wall time: 663 ms


In [56]:
m = RandomForestRegressor(n_estimators=20, min_samples_leaf=100, n_jobs=-1)
%time m.fit(x, y)

CPU times: user 3min 7s, sys: 46.9 ms, total: 3min 7s
Wall time: 1min 34s


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=100, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=-1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [57]:
print_score(m)

[0.2755978284934819, 0.2825882837845608, 0.2091973121808461, 0.1663215907637169]


In [58]:
m = RandomForestRegressor(n_estimators=20, min_samples_leaf=10, n_jobs=-1)
%time m.fit(x, y)
print_score(m)

CPU times: user 4min 3s, sys: 64.5 ms, total: 4min 4s
Wall time: 2min 2s
[0.21480555727835826, 0.27923860092421304, 0.5195949324892778, 0.1859686004319775]


In [60]:
m = RandomForestRegressor(n_estimators=20, min_samples_leaf=3, n_jobs=2)
%time m.fit(x, y)
print_score(m)

CPU times: user 4min 34s, sys: 153 ms, total: 4min 35s
Wall time: 2min 18s
[0.13157912419584467, 0.28526866715043125, 0.8197436649452858, 0.1504315097745349]
